In [2]:
import pandas as pd
import requests


# In[3]:


url = "https://api.pbpstats.com/get-totals/nba"
params = {
    "Season": "2023-24",
    "SeasonType": "Regular Season",
    "Type": "Player"
}
response = requests.get(url, params=params)
response_json = response.json()
player_stats = response_json["multi_row_table_data"]
df = pd.DataFrame(player_stats)

col = ['Name','Minutes','Points','FG2M', 'FG2A', 'FG3M', 'FG3A', 'TsPct','AssistPoints','AtRimAssists','ShortMidRangeAssists', 'LongMidRangeAssists','Corner3Assists','Arc3Assists','LostBallSteals', 'LiveBallTurnovers', 'BadPassOutOfBoundsTurnovers', 'BadPassTurnovers',
       'DeadBallTurnovers', 'LostBallOutOfBoundsTurnovers', 'LostBallTurnovers', 'StepOutOfBoundsTurnovers', 'Travels', 'Turnovers','OffensiveGoaltends','FTA','OffPoss','PtsAssisted2s',
'PtsUnassisted2s',
'PtsAssisted3s',
'PtsUnassisted3s',
 'DefPoss','TotalPoss']
df[col].to_csv('wowy/player_small.csv',index = False)
#df[col].to_csv('2024/player_tracking/player_small.csv',index = False)
df.to_csv('wowy/player_large.csv',index = False)
year = 2024
df['year'] = year
shotzone = df[['Name','EntityId','TeamId','TeamAbbreviation','GamesPlayed','OffPoss','DefPoss','Minutes','FtPoints','FTA','AtRimFGA','AtRimFGM','AtRimAccuracy','ShortMidRangeFGA','ShortMidRangeFGM',
        'ShortMidRangeAccuracy','ShortMidRangeFrequency','LongMidRangeFGM','LongMidRangeFGA','FG2A','FG2M','FG3A','NonHeaveFg3Pct','NonHeaveArc3FGA','HeaveAttempts','Corner3FGA','Corner3FGM','NonHeaveFg3Pct'
                ,'NonHeaveArc3FGM','TsPct','Points','EfgPct','SecondChanceEfgPct','PenaltyEfgPct','SecondChanceTsPct','PenaltyTsPct','SecondChanceShotQualityAvg','PenaltyShotQualityAvg','ShotQualityAvg','year']]

#shotzone
old = pd.read_csv('shotzone.csv')

old = old.rename(columns = {'NonHeaveFg3Pct.1':'NonHeaveFg3Pct'})

old = old[old.year<year]
shots = []
shots.append(old)
shots.append(shotzone)
master = pd.concat(shots)
master.to_csv('shotzone.csv',index = False)

url = "https://api.pbpstats.com/get-totals/nba"
params = {
    "Season": "2023-24",
    "SeasonType": "Playoffs",
    "Type": "Player"
}
response = requests.get(url, params=params)
response_json = response.json()
player_stats = response_json["multi_row_table_data"]
df = pd.DataFrame(player_stats)

col = ['Name','Minutes','Points','FG2M', 'FG2A', 'FG3M', 'FG3A', 'TsPct','AssistPoints','AtRimAssists','ShortMidRangeAssists', 'LongMidRangeAssists','Corner3Assists','Arc3Assists','LostBallSteals', 'LiveBallTurnovers', 'BadPassOutOfBoundsTurnovers', 'BadPassTurnovers',
       'DeadBallTurnovers', 'LostBallOutOfBoundsTurnovers', 'LostBallTurnovers', 'StepOutOfBoundsTurnovers', 'Travels', 'Turnovers','OffensiveGoaltends','FTA','OffPoss','PtsAssisted2s',
'PtsUnassisted2s',
'PtsAssisted3s',
'PtsUnassisted3s',
 'DefPoss','TotalPoss']
df[col].to_csv('wowy/player_small_ps.csv',index = False)
#df[col].to_csv('2024/player_tracking/player_small.csv',index = False)
df.to_csv('wowy/player_large_ps.csv',index = False)
year = 2024
df['year'] = year
shotzone = df[['Name','EntityId','TeamId','TeamAbbreviation','GamesPlayed','OffPoss','DefPoss','Minutes','FtPoints','FTA','AtRimFGA','AtRimFGM','AtRimAccuracy','ShortMidRangeFGA','ShortMidRangeFGM',
        'ShortMidRangeAccuracy','ShortMidRangeFrequency','LongMidRangeFGM','LongMidRangeFGA','FG2A','FG2M','FG3A','NonHeaveFg3Pct','NonHeaveArc3FGA','HeaveAttempts','Corner3FGA','Corner3FGM','NonHeaveFg3Pct'
                ,'NonHeaveArc3FGM','TsPct','Points','EfgPct','SecondChanceEfgPct','PenaltyEfgPct','SecondChanceTsPct','PenaltyTsPct','SecondChanceShotQualityAvg','PenaltyShotQualityAvg','ShotQualityAvg','year']]

#shotzone
old = pd.read_csv('shotzone_ps.csv')

old = old.rename(columns = {'NonHeaveFg3Pct.1':'NonHeaveFg3Pct'})

old = old[old.year<year]
shots = []
shots.append(old)
shots.append(shotzone)
master = pd.concat(shots)
master.to_csv('shotzone_ps.csv',index = False)
print('ShotZone Player')

ShotZone Player


In [7]:
poss = pd.read_csv('shotzone.csv')
poss = poss[['EntityId','OffPoss','DefPoss','year']]
poss.rename(columns={'EntityId':'PLAYER_ID'},inplace=True)
poss['Poss'] = poss['OffPoss']+poss['DefPoss']
poss.to_csv('poss.csv',index=False)


poss = pd.read_csv('shotzone_ps.csv')
poss = poss[['EntityId','OffPoss','DefPoss','year']]
poss.rename(columns={'EntityId':'PLAYER_ID'},inplace=True)
poss['Poss'] = poss['OffPoss']+poss['DefPoss']
poss.to_csv('poss_ps.csv',index=False)

In [2]:
def team_shotzone(year,ps = False):
    stype="Playoffs"
    if ps == False:
        stype="Regular Season"
    season = str(year)+'-'+str(year+1)[-2:]

                               
        
    url = "https://api.pbpstats.com/get-totals/nba"
    params = {
        "Season": season,
        "SeasonType": stype,
        "Type": "Team"
    }
    response = requests.get(url, params=params)
    response_json = response.json()
    player_stats = response_json["multi_row_table_data"]
    df = pd.DataFrame(player_stats)
    shotzone = df[['Name','EntityId','TeamId','TeamAbbreviation','GamesPlayed','OffPoss','DefPoss','FtPoints','FTA','AtRimFGA','AtRimFGM','AtRimAccuracy','ShortMidRangeFGA','ShortMidRangeFGM',
            'ShortMidRangeAccuracy','ShortMidRangeFrequency','LongMidRangeFGM','LongMidRangeFGA','FG2A','FG2M','FG3A','NonHeaveFg3Pct','NonHeaveArc3FGA','HeaveAttempts','Corner3FGA','Corner3FGM'
                    ,'NonHeaveArc3FGM','TsPct','Points','EfgPct','SecondChanceEfgPct','PenaltyEfgPct','SecondChanceTsPct','PenaltyTsPct','SecondChanceShotQualityAvg','PenaltyShotQualityAvg','ShotQualityAvg']].reset_index(drop=True)
    shotzone['year']=year+1
    shotzone = shotzone.reset_index(drop=True)
    return shotzone


def update_team(year,ps=False):
    trail = ''
    if ps == True:
        trail='_ps'
    teamdf = team_shotzone(year,ps=ps)
    print(teamdf.columns)
    year+=1
    old_df =pd.read_csv('team_shotzone'+trail+'.csv')
    print(old.columns)
    old_df = old_df[old_df.year!=year].reset_index(drop=True)
    new=pd.concat([old_df,teamdf])
    new.to_csv('team_shotzone'+trail+'.csv',index=False)
    return new
newdf = update_team(2023,ps=True)
newdf

Index(['Name', 'EntityId', 'TeamId', 'TeamAbbreviation', 'GamesPlayed',
       'OffPoss', 'DefPoss', 'FtPoints', 'FTA', 'AtRimFGA', 'AtRimFGM',
       'AtRimAccuracy', 'ShortMidRangeFGA', 'ShortMidRangeFGM',
       'ShortMidRangeAccuracy', 'ShortMidRangeFrequency', 'LongMidRangeFGM',
       'LongMidRangeFGA', 'FG2A', 'FG2M', 'FG3A', 'NonHeaveFg3Pct',
       'NonHeaveArc3FGA', 'HeaveAttempts', 'Corner3FGA', 'Corner3FGM',
       'NonHeaveArc3FGM', 'TsPct', 'Points', 'EfgPct', 'SecondChanceEfgPct',
       'PenaltyEfgPct', 'SecondChanceTsPct', 'PenaltyTsPct',
       'SecondChanceShotQualityAvg', 'PenaltyShotQualityAvg', 'ShotQualityAvg',
       'year'],
      dtype='object')
Index(['Name', 'EntityId', 'TeamId', 'TeamAbbreviation', 'GamesPlayed',
       'OffPoss', 'DefPoss', 'Minutes', 'FtPoints', 'FTA', 'AtRimFGA',
       'AtRimFGM', 'AtRimAccuracy', 'ShortMidRangeFGA', 'ShortMidRangeFGM',
       'ShortMidRangeAccuracy', 'ShortMidRangeFrequency', 'LongMidRangeFGM',
       'LongMidRangeFGA'

,Name,EntityId,TeamId,TeamAbbreviation,GamesPlayed,OffPoss,DefPoss,FtPoints,FTA,AtRimFGA,...,Points,EfgPct,SecondChanceEfgPct,PenaltyEfgPct,SecondChanceTsPct,PenaltyTsPct,SecondChanceShotQualityAvg,PenaltyShotQualityAvg,ShotQualityAvg,year
0,DAL,1610612742,1610612742,DAL,10,885,884,217,256,223,...,922,0.436262,0.440945,0.467980,0.482143,0.563218,0.508350,0.451904,0.480766,2001
1,LAL,1610612747,1610612747,LAL,16,1487,1488,332,491,430,...,1654,0.504966,0.568182,0.533784,0.576596,0.572785,0.509282,0.474461,0.483483,2001
2,MIL,1610612749,1610612749,MIL,18,1606,1606,341,426,303,...,1722,0.476207,0.469048,0.485337,0.512658,0.568765,0.475982,0.447151,0.451933,2001
3,MIN,1610612750,1610612750,MIN,4,349,352,70,86,62,...,328,0.405660,0.441860,0.386364,0.480392,0.505814,0.469317,0.449298,0.452846,2001
4,ORL,1610612753,1610612753,ORL,4,386,384,67,98,106,...,411,0.479109,0.468085,0.347826,0.490566,0.431818,0.500244,0.454658,0.468729,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,CLE,1610612739,1610612739,CLE,5,468,468,76,100,125,...,469,0.485185,0.513514,0.519531,0.558140,0.562069,0.536281,0.538014,0.554344,2024
12,IND,1610612754,1610612754,IND,5,469,471,61,90,97,...,558,0.544956,0.465517,0.472477,0.522388,0.536290,0.492235,0.530154,0.525640,2024
13,ORL,1610612753,1610612753,ORL,5,468,468,88,120,124,...,505,0.504843,0.471698,0.476974,0.516129,0.511628,0.547705,0.541533,0.548104,2024
14,PHI,1610612755,1610612755,PHI,5,461,459,102,125,127,...,534,0.523002,0.418367,0.458824,0.465517,0.541667,0.538062,0.535019,0.551406,2024


In [4]:
#df.to_csv('shotzone.csv',index = False)